# Split 3

Section 4.3 of lab1b

## Question 1

1. Create time series dataset

In [1]:
import numpy as np
from tools import mackey_glass_time_series

steps = np.arange(301, 1501)
x = mackey_glass_time_series(np.max(steps)+5)
input = np.array([[x[t-21], x[t-16], x[t-11], x[t-6], x[t-1]] for t in steps])
output = np.array([x[t+4] for t in steps])

2. Plot data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(context="notebook", style="whitegrid")

sns.lineplot(x=steps+5, y=output)
plt.xlabel("t")
plt.ylabel("Mackey-Glass value")
plt.savefig("imgs/time_series.pdf")

3. Split into training, validation, and test sets

In [3]:
X_train = input[:-400]
X_val = input[-400:-200]
X_test = input[-200:]

y_train = output[:-400]
y_val = output[-400:-200]
y_test = output[-200:]

The validation subset will be determined through the cross validation scheme.

## Question 2

In [4]:
from tqdm.notebook import tqdm
from tools import TimeSeriesMLP
import pandas as pd

def parameter_search(X_train, y_train, X_val, y_val, X_test, y_test, layers_config=None, early_stopping=True, n_epochs=500, lambdas=None):
    histories = {}

    lc = tqdm(layers_config if lambdas is  None else lambdas)
    for item in lc:
        lc.set_description_str(f"Evaluating {item}...")

        history = {
        "train_loss": [],
        "val_loss": [],
        "test_loss": [],
        "best_loss": [],
        "epoch": [],
    }
        for _ in tqdm(range(10), leave=False):
            if lambdas is None:
                model = TimeSeriesMLP(hidden_nodes=item, early_stopping=early_stopping, n_epochs=n_epochs) 
            else:
                model = TimeSeriesMLP(hidden_nodes=layers_config, lmbda=item, early_stopping=False, n_epochs=n_epochs)
            h = model.fit(X_train, y_train, X_val, y_val)
            history["train_loss"] += h["loss"]
            history["val_loss"] += h["val_loss"]
            history["test_loss"] += [model.evaluate(X_test, y_test)] * len(h["loss"])
            history["best_loss"] += [h["val_loss"][-1]] * len(h["loss"])
            history["epoch"] += (np.arange(len(h["loss"])) + 1).tolist()

        history = pd.DataFrame(history)
        history = history.groupby("epoch").agg(["mean", "std"])
        histories[item] = history.reset_index().to_dict("list")

    return histories

In [ ]:
from tools import plot_training_histories
from itertools import product

from warnings import filterwarnings
import pandas as pd

filterwarnings("ignore", category=UserWarning)

histories = {}
layers_config = list(product([3, 4, 5], [2, 4, 6]))

histories = parameter_search(X_train, y_train, X_val, y_val, X_test, y_test, layers_config=layers_config)

fig, _ = plot_training_histories(histories)
fig.savefig("imgs/layers_selection.pdf")

In [ ]:
final_val_errors = {
    config: (history[('best_loss', 'mean')][-1], history[('best_loss', 'std')][-1])
    for config, history in histories.items()
}

ranked_by_final = sorted(
    final_val_errors.items(),
    key=lambda x: x[1][0],
)

print("Rankings by final validation error:")
for i, (config, error) in enumerate(ranked_by_final, 1):
    print(f"{i}. Architecture {config}: {error[0]:.4f} (std = {error[1]:.4f})")

Rankings by final validation error:
1. Architecture (4, 4): 0.0012 (std = 0.0004)
2. Architecture (4, 2): 0.0012 (std = 0.0003)
3. Architecture (3, 6): 0.0012 (std = 0.0006)
4. Architecture (5, 6): 0.0013 (std = 0.0007)
5. Architecture (5, 2): 0.0014 (std = 0.0002)
6. Architecture (5, 4): 0.0014 (std = 0.0008)
7. Architecture (3, 4): 0.0017 (std = 0.0004)
8. Architecture (4, 6): 0.0018 (std = 0.0010)
9. Architecture (3, 2): 0.0019 (std = 0.0009)

## Question 3

In [ ]:
best_test = (histories[ranked_by_final[0][0]][("test_loss", "mean")][-1], histories[ranked_by_final[0][0]][("test_loss", "std")][-1])
worst_test = (histories[ranked_by_final[-1][0]][("test_loss", "mean")][-1], histories[ranked_by_final[-1][0]][("test_loss", "std")][-1])

print(f"Best model performance on test set: {best_test[0]:.4f} (std = {best_test[1]:.4f})")
print(f"Worst model performance on test set: {worst_test[0]:.4f} (std = {worst_test[1]:.4f})")

Best model performance on test set: 0.0009518225560896099

Worst model performance on test set: 0.0011794989695772529

In [ ]:
best_model = TimeSeriesMLP(hidden_nodes=ranked_by_final[0][0])
best_model.fit(X_train, y_train, X_val, y_val)
worst_model = TimeSeriesMLP(hidden_nodes=ranked_by_final[-1][0])
worst_model.fit(X_train, y_train, X_val, y_val)

data = {
    "t": steps[-200:]+5,
    "Truth": y_test,
    "Best Model": best_model.predict(X_test).flatten(),
    "Worst Model":worst_model.predict(X_test).flatten()
}
data = pd.DataFrame(data)
data = pd.melt(data, ["t"])
data.columns = ["t", "Model", "Value"]

sns.lineplot(data, x="t", y="Value", hue="Model")

plt.title('Actual vs Predicted Values')
plt.legend()

plt.savefig("imgs/time_series_best_vs_worst.pdf")

## Question 4

In [ ]:
sigmas = [0.05, 0.15]

noised_inputs = []
noised_outputs = []

fig, axs = plt.subplots(1, 2, figsize=(14, 5))

fig.suptitle("Noisy Time Series given a Standard Deviation")

for sigma, ax in zip(sigmas, axs):
    length = np.max(steps)+5
    x = mackey_glass_time_series(length) + np.random.normal(0, sigma, length)
    input = np.array([[x[t-21], x[t-16], x[t-11], x[t-6], x[t-1]] for t in steps])
    output = np.array([x[t+4] for t in steps])

    noised_inputs.append(input)
    noised_outputs.append(output)

    sns.lineplot(x=steps+5, y=output, ax=ax)

    ax.set_title(f"$\\sigma = {sigma}$")
    ax.set_xlabel("t")
    ax.set_ylabel("Value")

plt.tight_layout()
plt.savefig(f"imgs/time_series_noise.pdf")

In [ ]:
from itertools import product
from tqdm.notebook import tqdm
from warnings import filterwarnings

filterwarnings("ignore", category=UserWarning)

layers_config = list(product([4], [3, 6, 9]))
sigma_histories = {}

iterator = tqdm(sigmas)

for sigma in iterator:
    iterator.set_description(f"Running with sigma = {sigma}")

    Xn_train = X_train + np.random.normal(0, sigma, X_train.shape)
    yn_train = y_train + np.random.normal(0, sigma, y_train.shape)

    sigma_histories[sigma] = parameter_search(
        Xn_train,
        yn_train,
        X_val,
        y_val,
        X_test,
        y_test,
        layers_config=layers_config,
        early_stopping=False,
        n_epochs=200
    )

In [ ]:
from tools import plot_training_histories

for sigma, histories in sigma_histories.items():
    fig, ax = plot_training_histories(histories)
    plt.title(f"Training History for $\\sigma = {sigma}$")
    plt.savefig(f"imgs/layers_selection_{sigma}_noisy.pdf")

In [ ]:
best_configs = {0.05: (4, 6), 0.15: (4, 9)}
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

fig.suptitle("Actual vs Predicted Values")

for (sigma, config), ax in zip(best_configs.items(), axs):
    model = TimeSeriesMLP(
        hidden_nodes=config, n_epochs=200, early_stopping=False, lmbda=0
    )
    model.fit(
        X_train + np.random.normal(loc=0, scale=sigma, size=X_train.shape),
        y_train + np.random.normal(loc=0, scale=sigma, size=y_train.shape),
        X_val,
        y_val,
    )
    data = {
        "t": steps[-200:] + 5,
        "Truth": y_test,
        f"{config}": model.predict(X_test).flatten(),
    }
    data = pd.DataFrame(data)
    data = pd.melt(data, ["t"])
    data.columns = ["t", "Model", "Value"]

    ax = sns.lineplot(data, x="t", y="Value", hue="Model", ax=ax)

    ax.set_title(f"$\\sigma = {sigma}$")
    ax.legend()

plt.savefig(f"imgs/time_series_noisy_test.pdf")

In [ ]:
best_histories = {
    (sigma, best_configs[sigma]): histories[best_configs[sigma]]
    for sigma, histories in sigma_histories.items()
}

for (sigma, config), history in best_histories.items():
    best_test = history[("test_loss", "mean")][-1]

    print(f"{config} model performance on test set (SD = {sigma}): {best_test:0.4f}")

In [ ]:
lambdas = [1e-4, 1e-3, 5e-3, 1e-2]
lambda_histories = {}
models = {}

iterator = tqdm(best_configs.items())

for (sigma, config) in iterator:
    iterator.set_description(f"Running with sigma = {sigma}")
    Xn_train = X_train + np.random.normal(loc = 0, scale=sigma, size=X_train.shape)
    Xn_val = X_val
    Xn_test = X_test

    yn_train = y_train + np.random.normal(loc=0, scale=sigma, size=y_train.shape)
    yn_val = y_val
    yn_test = y_test

    lambda_histories[sigma] = parameter_search(
        Xn_train,
        yn_train,
        Xn_val,
        yn_val,
        Xn_test,
        yn_test,
        layers_config=config,
        lambdas=lambdas,
        n_epochs=200,
    )

In [ ]:
for sigma, histories in lambda_histories.items():
    fig, ax = plot_training_histories(histories, type="lambda")
    plt.title(f"Training History for $\\sigma = {sigma}$")
    plt.savefig(f"imgs/layers_selection_{sigma}_noisy_lambda.pdf")

In [ ]:
lambdas = [1e-4, 1e-3, 5e-3, 1e-2]
best_configs = {0.05: (4, 6), 0.15: (4, 9)}
fig, axs = plt.subplots(2, 4, figsize=(12, 10))

fig.suptitle("Actual vs Predicted Values")

for i, (sigma, config) in enumerate(best_configs.items()):
    for j, lmbda in enumerate(lambdas):
        model = TimeSeriesMLP(
            hidden_nodes=config, n_epochs=200, early_stopping=False, lmbda=lmbda
        )
        model.fit(
            X_train + np.random.normal(loc=0, scale=sigma, size=X_train.shape),
            y_train + np.random.normal(loc=0, scale=sigma, size=y_train.shape),
            X_val,
            y_val,
        )
        data = {
            "t": steps[-200:] + 5,
            "Truth": y_test,
            f"{config}": model.predict(X_test).flatten(),
        }
        data = pd.DataFrame(data)
        data = pd.melt(data, ["t"])
        data.columns = ["t", "Model", "Value"]

        axs[i][j] = sns.lineplot(data, x="t", y="Value", hue="Model", ax=axs[i][j])

        axs[i][j].set_title(f"$\\sigma = {sigma}$, $\\lambda = {lmbda}$")
        axs[i][j].legend()

plt.tight_layout()
plt.savefig(f"imgs/time_series_noisy_test_lambda.pdf")

In [ ]:
from tools import TimeSeriesMLP

fig, axs = plt.subplots(2, 4, figsize=(12, 10))

fig.suptitle("Weight Distribution for each Model")

for i, ((sigma, config), input, output) in enumerate(zip(best_configs.items(), noised_inputs, noised_outputs)):
    for j, lmbda in enumerate(lambdas):
        model = TimeSeriesMLP(hidden_nodes=config, n_epochs=200, early_stopping=False, lmbda=lmbda)
        model.fit(input[:-280], output[:-280], input[-280:-200], output[-280:-200])
        
        data = []
        for w in model.get_weights():
            data.extend(w.flatten())

        axs[i][j] = sns.histplot(data, ax=axs[i][j])

        axs[i][j].set_title(f'$\\sigma = {sigma}$, $\\lambda = {lmbda}$')
        axs[i][j].set_xlabel("Weight Value")
        axs[i][j].set_ylabel("Frequency")

plt.tight_layout()
plt.savefig("./imgs/weights_distribution.pdf")